In [ ]:
!nvidia-smi

Wed Sep 22 05:22:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Colab/kaggle/kaggle.json ~/.kaggle

!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 5.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=a4be8569d753451f38ee99fde18403035424d2f63688440b61dee509407f2bde
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!cp -r /content/drive/MyDrive/Colab/kaggle/chaii-hindi-and-tamil-question-answering/input /content
!mkdir /content/checkpoint

In [ ]:
# リロード必須
!pip install -U pytorch-lightning transformers wandb sentencepiece torchsummaryX

     |████████████████████████████████| 923 kB 15.5 MB/s 
     |████████████████████████████████| 2.8 MB 72.0 MB/s 
     |████████████████████████████████| 1.7 MB 69.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.1 MB/s 
     |████████████████████████████████| 282 kB 88.3 MB/s 
     |████████████████████████████████| 636 kB 62.7 MB/s 
     |████████████████████████████████| 119 kB 84.4 MB/s 
     |████████████████████████████████| 829 kB 53.5 MB/s 
     |████████████████████████████████| 1.3 MB 40.4 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 42.8 MB/s 
     |████████████████████████████████| 895 kB 85.7 MB/s 
     |████████████████████████████████| 139 kB 90.8 MB/s 
     |████████████████████████████████| 97 kB 9.0 MB/s 
     |████████████████████████████████| 180 kB 53.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 294 kB 76.3 MB/s 
     |██████████████

In [ ]:
import os
import random
from dataclasses import dataclass

import numpy as np
import pandas as pd
import math
import sklearn.model_selection as sms

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from torchsummaryX import summary

from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm

import json
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi

## config

In [ ]:
@dataclass
class Config:
    #########################
    # Globals #
    #########################
    exp_name = "exp004"
    debug = False
    gpus = 1
    num_workers = 8
    num_epochs = 5
    grad_accumulate = 2
    fp16 = False
    seed = [42, 2434, 98, 24, 1991]
    #########################
    # Data #
    #########################
    train_csv = "input/train.csv"
    test_csv = "input/test.csv"
    sample_submission_csv = "input/sample_submission.csv"
    mlqa_hindi_csv = "input/external_data/mlqa_hindi.csv"
    xquad_hindi_csv = "input/external_data/xquad_hindi.csv"
    checkpoint_dir = "checkpoint"
    #########################
    # Split #
    #########################
    split_name = "StratifiedKFold"
    split_params = {
        "n_splits": 5,
        "random_state": 42,
        "shuffle": True,
        }
    #########################
    # Tokenizer #
    #########################
    tokenizer_name = "deepset/xlm-roberta-large-squad2"
    max_seq_length = 384
    doc_stride = 128
    truncation = "only_second"
    padding = "max_length"
    #########################
    # Dataset #
    #########################
    #########################
    # DataLoader #
    #########################
    train_batch_size = 4
    valid_batch_size = 8
    test_batch_size = 128
    #########################
    # Model #
    #########################
    base_model_name = "deepset/xlm-roberta-large-squad2"
    base_model_config = "deepset/xlm-roberta-large-squad2"
    num_classes = 2
    init_layers = 1
    #########################
    # Criterion #
    #########################
    loss_name = "CrossEntropyLoss"
    loss_params = {
        "ignore_index": -1,
    }
    #########################
    # Optimizer #
    #########################
    optimizer_name = "AdamW"
    optimizer_params = {
        "lr": 1.5e-5,
        "weight_decay": 1e-2,
        "eps": 1e-8,
        "correct_bias": True
    }
    no_decay = ["bias", "LayerNorm.weight"]
    header_weight_decay = 1e-2
    header_lr = 1.e-3
    lr_decay = 0.98
    ######################
    # Scheduler #
    ######################
    scheduler_name = "linear-warmup"
    scheduler_params = {
        "warmup_ratio": 0.1,
    }
    ######################
    # Callbacks #
    ######################
    model_checkpoint_params = {
        "monitor": "val/loss_epoch",
        "save_top_k": 1,
        "save_weights_only": True,
    }
    early_stopping_params = {
        "monitor": "val/loss_epoch",
        "min_delta": 0.0,
        "patience": 5,
        "verbose": False,
        "mode": "min",
    }
    wandb_logger_params = {
        "project": "kaggle-chaii-hindi-and-tamil-question-answering"
    }


cfg = Config()

## utils

In [ ]:
def load_dataset(cfg):
    train_df = pd.read_csv(cfg.train_csv, nrows= 100 if cfg.debug else None)
    test_df = pd.read_csv(cfg.test_csv)
    external_mlqa_df = pd.read_csv(cfg.mlqa_hindi_csv)
    external_xquad_df = pd.read_csv(cfg.xquad_hindi_csv)
    external_train_df = pd.concat([external_mlqa_df, external_xquad_df], axis=0).reset_index(drop=True)
    sample_submission_df = pd.read_csv(cfg.sample_submission_csv)

    return train_df, test_df, external_train_df, sample_submission_df


def get_split(cfg):
    split_name = cfg.split_name
    split_params = cfg.split_params

    return sms.__getattribute__(split_name)(**split_params)


def get_fold(cfg, train_df: pd.DataFrame, y_train: pd.DataFrame):
    splitter = get_split(cfg)
    train_df["fold"] = -1
    for fold_id, (train_idx, valid_idx) in enumerate(splitter.split(train_df, y_train)):
        train_df.loc[valid_idx, "fold"] = int(fold_id)

    return train_df

## load data

In [ ]:
train_df, test_df, external_train_df, sample_submission_df = load_dataset(cfg)

In [ ]:
train_df = get_fold(cfg, train_df, train_df["language"])
external_train_df["fold"] = -1
external_train_df["id"] = list(np.arange(1, len(external_train_df) + 1))
train_df = pd.concat([train_df, external_train_df], axis=0).reset_index(drop=True)
train_df

,id,context,question,answer_text,answer_start,language,fold
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,0
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,0
...,...,...,...,...,...,...,...
7724,6611,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,-1
7725,6612,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,संरचनाओं में तनाव का कारण क्या बनता है?,तनाव टेंसर,343,hindi,-1
7726,6613,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,किसी वस्तु के आयतन में क्रॉस सेक्शन क्षेत्र की...,दबाव की शर्तें,118,hindi,-1
7727,6614,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,सामान्य ताकतों से क्या जुड़ा है?,दबाव की शर्तें,118,hindi,-1


## preprocess

In [ ]:
def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}

train_df["answers"] = train_df[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
train_df.head()

,id,context,question,answer_text,answer_start,language,fold,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,0,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,0,"{'answer_start': [585], 'text': ['சூரியனும்']}"


In [ ]:
# # トークナイザーの準備
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# batch_sentences = ["Hello I'm a single sentence","And another sentence","And the very very last one"]
# batch_of_second_sentences = ["I'm a sentence that goes with the first sentence","And I should be encoded with the second sentence","And I go with the very last one"]

# encoded_inputs = tokenizer(
#     batch_sentences,
#     batch_of_second_sentences,
#     truncation="only_second",
#     max_length=20,
#     stride=2,
#     return_overflowing_tokens=True,
#     return_offsets_mapping=True,
#     padding="max_length"
# )

# print(encoded_inputs)

In [ ]:
def prepare_train_features(cfg, example, tokenizer):
    example["question"] = example["question"].lstrip()

    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation=cfg.truncation,
        max_length=cfg.max_seq_length,
        stride=cfg.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=cfg.padding,
        # return_tensors="pt"
    )

    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_example.pop("offset_mapping")

    features = []
    for i, offsets in enumerate(offset_mapping):
        feature = {}

        input_ids  = tokenized_example["input_ids"][i]
        attention_mask = tokenized_example["attention_mask"][i]

        feature["input_ids"] = input_ids
        feature["attention_mask"] = attention_mask
        feature["offset_mapping"] = offsets

        cls_index = input_ids.index(tokenizer.cls_token_id) # cls_token_id = 101, pad_token_id = 0
        sequence_ids = tokenized_example.sequence_ids(i)

        # sample_indx = sample_mapping[i]
        answers = example["answers"]

        if len(answers["answer_start"]) == 0: # sampleにanswerがなければ開始・終了位置をCLS(=[CLS], <s>)にする
            feature["start_position"] = cls_index
            feature["end_position"] = cls_index
        else:
            start_char = answers["answer_start"][0] # answer部分の文字レベルの開始位置
            end_char = start_char + len(answers["text"][0]) # answer部分の文字レベルの終了位置

            token_start_index = 0 # sequenceにおけるcontext部分のトークンレベルの開始位置
            while sequence_ids[token_start_index] != 1: # sequence_idsが1(=context部分)になるまで足す(<s>,</s>, querstion部分を飛ばすイメージ)
                token_start_index += 1

            token_end_index = len(input_ids) - 1 # sequenceにおけるcontext部分のトークンレベルの終了位置
            while sequence_ids[token_end_index] != 1: # sequence_idsが1(=context部分)になるまで引く(paddingはNoneなので<pad>部分を引くイメージ)
                token_end_index -= 1

            # token_start_index, token_end_indexがanswer部分の範囲外にあればpositionを0(=[CLS], <s>)とする(truncationされてる場合はこっち)
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            # star_char, end_charを含むtokenまでtoken_start_index, token_end_indexを調整
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                feature["start_position"] = token_start_index - 1 # 行き過ぎた1token分戻す

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                feature["end_position"] = token_end_index + 1 # 行き過ぎた1token分戻す

        features.append(feature)

    return features


def prepare_test_features(cfg, example, tokenizer):
    example["question"] = example["question"].strip()

    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation=cfg.truncation,
        max_length=cfg.max_seq_length,
        stride=cfg.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=cfg.padding,
        # return_tensors="pt"
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example["id"]
        feature["context"] = example["context"]
        feature["question"] = example["question"]
        feature["input_ids"] = tokenized_example["input_ids"][i]
        feature["attention_mask"] = tokenized_example["attention_mask"][i]
        feature["offset_mapping"] = tokenized_example["offset_mapping"][i]
        feature["sequence_ids"] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)

    return features

## Dataset

In [ ]:
class ChaiiDataset(Dataset):
    def __init__(self, features, phase):
        super(ChaiiDataset, self).__init__()
        self.features = features
        self.phase = phase
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        feature = self.features[index]

        if self.phase == "train":
            return {
                "input_ids": torch.tensor(feature["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(feature["attention_mask"], dtype=torch.long),
                "offset_mapping": torch.tensor(feature["offset_mapping"], dtype=torch.long),
                "start_position": torch.tensor(feature["start_position"], dtype=torch.long),
                "end_position": torch.tensor(feature["end_position"], dtype=torch.long),
            }
        else:
            return {
                "input_ids": torch.tensor(feature["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(feature["attention_mask"], dtype=torch.long),
                "offset_mapping": torch.tensor(feature["offset_mapping"], dtype=torch.long),
                "sequence_ids": feature["sequence_ids"],
                "id": feature["example_id"],
                "context": feature["context"],
                "question": feature["question"],
            }


class ChaiiDataModule(pl.LightningDataModule):
    def __init__(self, cfg, tokenizer, input_df: pd.DataFrame, phase: str, fold: int = 0):
        super(ChaiiDataModule, self).__init__()
        self.cfg = cfg
        self.tokenizer = tokenizer
        self.input_df = input_df
        self.phase = phase
        self.fold = fold

    def get_train_features(self, cfg, input_df, tokenizer):
        features = []
        for i, row in tqdm(input_df.iterrows(), total=len(input_df), desc="[get features]"):
            features += prepare_train_features(cfg, row, tokenizer)
        
        return features

    def get_test_features(self, cfg, input_df, tokenizer):
        features = []
        for i, row in tqdm(input_df.iterrows(), total=len(input_df), desc="[get features]"):
            features += prepare_test_features(cfg, row, tokenizer)
        
        return features

    def setup(self, stage=None):
        assert self.phase in ("train", "test"), "Input phase is not exist."
        if self.phase == "train":
            train_df = self.input_df[self.input_df["fold"] != self.fold].reset_index(drop=True)
            valid_df = self.input_df[self.input_df["fold"] == self.fold].reset_index(drop=True)

            self.train_features = self.get_train_features(self.cfg, train_df, self.tokenizer)
            self.valid_features = self.get_train_features(self.cfg, valid_df, self.tokenizer)

            self.train_dataset = ChaiiDataset(self.train_features, self.phase)
            self.valid_dataset = ChaiiDataset(self.valid_features, self.phase)
            print(f"Number of train features: {len(self.train_dataset)}, Number of valid features: {len(self.valid_dataset)}")
        elif self.phase == "test":
            self.test_features = self.get_test_features(self.cfg, self.input_df, self.tokenizer)
            self.test_dataset = ChaiiDataset(self.test_features, self.phase)
            print(f"Number of test features: {len(self.test_dataset)}")
        else:
            raise NotImplementedError
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.cfg.train_batch_size, num_workers=self.cfg.num_workers, pin_memory=True, shuffle=True, drop_last=False)

    def val_dataloader(self):
        return DataLoader(self.valid_dataset, batch_size=self.cfg.valid_batch_size, num_workers=self.cfg.num_workers, pin_memory=True, shuffle=False, drop_last=False)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.cfg.test_batch_size, num_workers=self.cfg.num_workers, pin_memory=True, shuffle=False, drop_last=False)

## Model

In [ ]:
class ChaiiModel(nn.Module):
    def __init__(self, cfg):
        super(ChaiiModel, self).__init__()
        self.cfg = cfg
        self.model_config = AutoConfig.from_pretrained(self.cfg.base_model_config)
        self.base_model = AutoModel.from_pretrained(self.cfg.base_model_name, config=self.model_config)
        # in_features = self.base_model.pooler.dense.in_features
        self.classifier = nn.Linear(self.model_config.hidden_size, cfg.num_classes)
        self._init_header_weights(self.classifier)
        # self._init_roberta_weights(self.base_model)

    def _init_header_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.model_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def _init_roberta_weights(self, model):
        for layer in model.encoder.layer[-cfg.init_layers:]:
            for module in layer.modules():
                if isinstance(module, nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                    module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
                    if module.padding_idx is not None:
                        module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)

    def forward(self, input_ids, attention_mask=None):
        output = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        x = self.classifier(output[0]) # (batch_size, hidden_size, num_classes)
        x0, x1 = x.split(1, dim=-1) # (batch_size, hidden_size, 1)
        start_logits, end_logits = x0.squeeze(-1), x1.squeeze(-1) # (batch_size, hidden_size)

        return start_logits, end_logits

## training

In [ ]:
def get_optimizer_grouped_parameters(cfg, model):
    no_decay = cfg.no_decay
    # header layerのweight_decay, lr
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if "classifier" in n],
            "weight_decay": cfg.header_weight_decay,
            "lr": cfg.header_lr
        },
    ]

    # num_layers = model.base_model.config.num_hidden_layers
    layers = [getattr(model, "base_model").embeddings] + list(getattr(model, "base_model").encoder.layer)
    layers.reverse()
    lr = cfg.optimizer_params["lr"]
    
    for layer in layers:
        lr *= cfg.lr_decay
        optimizer_grouped_parameters += [
            # no_decayのリストに含まれないパラメータはweight decayを設定
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": cfg.optimizer_params["weight_decay"],
                "lr": lr,
            },
            # no_decayのリストに含まれるパラメータはweight decayを設定しない
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            }
        ]
    
    return optimizer_grouped_parameters


def get_criterion(cfg):
    loss_name = cfg.loss_name
    loss_params = cfg.loss_params
    return nn.__getattribute__(loss_name)(**loss_params)


def get_optimizer(cfg, model=None, optimizer_grouped_parameters=None):
    optimizer_name = cfg.optimizer_name
    optimizer_params = cfg.optimizer_params

    # no_decay = ["bias", "LayerNorm.weight"]
    # optimizer_grouped_parameters = [
    #     {
    #         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    #         "weight_decay": cfg.optimizer_params["weight_decay"],
    #     },
    #     {
    #         "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
    #         "weight_decay": 0.0,
    #     },
    # ]

    if optimizer_name == "AdamW":
        return AdamW(
            optimizer_grouped_parameters,
            **optimizer_params
        )
    else:
        return optim.__getattribute__(optimizer_name)(model.parameters(), **optimizer_params)


def get_scheduler(cfg, optimizer, num_warmup_steps=None, num_training_steps=None):
    scheduler_name = cfg.scheduler_name
    scheduler_params = cfg.scheduler_params

    if scheduler_name == "cosine-warmup":
        return get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps,
        )
    elif scheduler_name == "linear-warmup":
        return get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps,
        )
    elif scheduler_name not in ("cosine-warmup", "linear-warmup"):
        return optim.lr_scheduler.__getattribute__(scheduler_name)(optimizer, **scheduler_params)
    elif scheduler_name is not None:
        return


class ChaiiLightningModule(pl.LightningModule):
    def __init__(self, cfg):
        super(ChaiiLightningModule, self).__init__()
        self.cfg = cfg
        self.model = ChaiiModel(self.cfg)
        self.criterion = get_criterion(self.cfg)

    def forward(self, input_ids, attention_mask):
        output_start, output_end = self.model(input_ids, attention_mask)
        return output_start, output_end

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, target_start, target_end = batch["input_ids"], batch["attention_mask"], batch["start_position"], batch["end_position"]
        output_start, output_end = self.forward(input_ids, attention_mask)

        loss_start = self.criterion(output_start, target_start)
        loss_end = self.criterion(output_end, target_end)
        loss = (loss_start + loss_end) / 2
        self.log("train/loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train/loss_start", loss_start, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train/loss_end", loss_end, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, target_start, target_end = batch["input_ids"], batch["attention_mask"], batch["start_position"], batch["end_position"]
        output_start, output_end = self.forward(input_ids, attention_mask)

        loss_start = self.criterion(output_start, target_start)
        loss_end = self.criterion(output_end, target_end)
        loss = (loss_start + loss_end) / 2
        self.log("val/loss", loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        # self.log("val/loss_start", loss_start, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        # self.log("val/loss_end", loss_end, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        outputs = {
            "loss": loss.detach(),
        }

        return outputs

    def validation_epoch_end(self, outputs):
        loss = torch.stack([output["loss"] for output in outputs]).mean().item()

        self.log("val/loss_epoch", loss, on_epoch=True, prog_bar=True, logger=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        input_ids, attention_mask = batch["input_ids"], batch["attention_mask"]
        output_start, output_end = self.forward(input_ids, attention_mask)

        returns = {
            "start_logits": output_start,
            "end_logits": output_end,
        }

        return returns

    def configure_optimizers(self):
        optimizer_grouped_parameters = get_optimizer_grouped_parameters(self.cfg, self.model)
        optimizer = get_optimizer(
            self.cfg,
            # self.model,
            optimizer_grouped_parameters=optimizer_grouped_parameters
        )
        num_training_steps = math.ceil(len(self.trainer.datamodule.train_dataloader()) / self.trainer.accumulate_grad_batches) * self.trainer.max_epochs
        if self.cfg.scheduler_params["warmup_ratio"] > 0:
            num_warmup_steps = int(num_training_steps * self.cfg.scheduler_params["warmup_ratio"])
        else:
            num_warmup_steps = 0
        scheduler = get_scheduler(self.cfg, optimizer, num_warmup_steps, num_training_steps)
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        print(f"Total Training Steps: {num_training_steps}, Total Warmup Steps: {num_warmup_steps}")

        return [optimizer], [scheduler]


def run_fold(cfg, train_df, fold, tokenizer):
    seed_everything(cfg.seed[0])

    checkpoint_callback = ModelCheckpoint(
        dirpath=cfg.checkpoint_dir,
        filename=f"{cfg.exp_name}_fold_{fold}",
        **cfg.model_checkpoint_params,
    )

    early_stopping_callback = EarlyStopping(**cfg.early_stopping_params)

    wandb_logger = WandbLogger(
        name=f"{cfg.exp_name}_fold_{fold}",
        **cfg.wandb_logger_params,
    )

    trainer = Trainer(
        default_root_dir=cfg.checkpoint_dir,
        gpus=cfg.gpus,
        max_epochs=cfg.num_epochs,
        accumulate_grad_batches=cfg.grad_accumulate,
        precision=16 if cfg.fp16 else 32,
        callbacks=[
            checkpoint_callback,
            # early_stopping_callback,
        ],
        logger=[
            wandb_logger,
        ],
        log_every_n_steps=10,
    )

    model = ChaiiLightningModule(cfg)
    datamodule = ChaiiDataModule(cfg, tokenizer=tokenizer, input_df=train_df, phase="train", fold=fold)
    trainer.fit(model, datamodule=datamodule)

    return checkpoint_callback.best_model_path

In [16]:
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

best_path = run_fold(cfg, train_df, 0, tokenizer)

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
ID = "azupero"
DATASET_ID = f"chaii-qa-checkpoint-{cfg.exp_name}"
UPLOAD_DIR = Path(cfg.checkpoint_dir)

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(UPLOAD_DIR / 'dataset-metadata.json', 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')

dataset_create_new()

Starting upload for file exp004_fold_0.ckpt


  0%|          | 0.00/2.09G [00:00<?, ?B/s]

## inference

In [ ]:
# def postprocess_qa_predictions(examples: pd.DataFrame, features, raw_predictions, n_best_size=20, max_answer_length=30):
#     '''予測値の後処理関数
#     '''
#     all_start_logits, all_end_logits = raw_predictions

#     example_id_to_index = {k: i for i, k in enumerate(examples["id"])} # dataframeのidをkeyとしたindexのmapping用dict
#     features_per_example = defaultdict(list) # dataframeのidに対応するexample_id_to_indexのidをkeyとしたdict。valueはfeaturesに対応するidのリスト
#     for i, feature in enumerate(features):
#         features_per_example[example_id_to_index[feature["example_id"]]].append(i)

#     predictions = OrderedDict()
#     print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

#     for example_index, example in examples.iterrows():
#         feature_indices = features_per_example[example_index] # 対応するfeaturesのindexを取り出す

#         # min_null_score = None
#         valid_answers = []

#         context = example["context"]
#         for feature_index in feature_indices:
#             start_logits = all_start_logits[feature_index]
#             end_logits = all_end_logits[feature_index]

#             sequence_ids = features[feature_index]["sequence_ids"]
#             context_index = 1 # contextのsequence_id(questionは0)
            
#             # contextのみoffset_mappingを保持(questionのoffset_mappingを除去)
#             features[feature_index]["offset_mapping"] = [
#                 (o if sequence_ids[k] == context_index else None)
#                 for k, o in enumerate(features[feature_index]["offset_mapping"])
#             ]
#             offset_mapping = features[feature_index]["offset_mapping"]
#             cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)

#             # feature_null_score = start_logits[cls_index] + end_logits[cls_index]
#             # if min_null_score is None or min_null_score < feature_null_score:
#             #     min_null_score = feature_null_score

#             # start_indexes = np.argsort(start_logits)[-1: -n_best_size - 1: -1].tolist()
#             # end_indexes = np.argsort(end_logits)[-1: -n_best_size - 1: -1].tolist()
#             start_indexes = np.argsort(start_logits)[::-1][:n_best_size].tolist()
#             end_indexes = np.argsort(end_logits)[::-1][:n_best_size].tolist()
#             for start_index in start_indexes:
#                 for end_index in end_indexes:
#                     if (
#                         start_index >= len(offset_mapping)
#                         or end_index >= len(offset_mapping)
#                         or offset_mapping[start_index] is None
#                         or offset_mapping[end_index] is None
#                     ):
#                         continue
#                     # Don't consider answers with a length that is either < 0 or > max_answer_length.
#                     if end_index < start_index or end_index - start_index + 1 > max_answer_length:
#                         continue

#                     start_char = offset_mapping[start_index][0]
#                     end_char = offset_mapping[end_index][1]
#                     valid_answers.append(
#                         {
#                             "score": start_logits[start_index] + end_logits[end_index],
#                             "text": context[start_char: end_char]
#                         }
#                     )
#         # 各レコード・チャンク(feature)におけるstart+end出力値のスコアが最も大きいペアを最終的な予測値とする
#         if len(valid_answers) > 0:
#             best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
#         else:
#             best_answer = {"text": "", "score": 0.0}

#         predictions[example["id"]] = best_answer["text"]

#     return predictions

In [ ]:
# def run_predict(cfg, tokenizer, input_df, checkpoint_path):
#     start_logits = []
#     end_logits = []

#     for path in checkpoint_path:
#         seed_everything(cfg.seed[0])

#         model = ChaiiLightningModule.load_from_checkpoint(checkpoint_path=path, cfg=cfg)
#         datamodule = ChaiiDataModule(cfg, tokenizer=tokenizer, input_df=input_df, phase="test")
#         trainer = Trainer(gpus=cfg.gpus)
#         preds = trainer.predict(model, datamodule=datamodule, return_predictions=True)
#         start_logits.append(torch.cat([pred["start_logits"] for pred in preds]).cpu().numpy())
#         end_logits.append(torch.cat([pred["end_logits"] for pred in preds]).cpu().numpy())

#     start_logits, end_logits = np.mean(start_logits, axis=0), np.mean(end_logits, axis=0)
#     predictions = postprocess_qa_predictions(input_df, datamodule.test_features, (start_logits, end_logits))

#     return predictions

In [ ]:
# checkpoint_path = ["checkpoint/exp001_fold_0.ckpt"]
# tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")
# predictions = run_predict(cfg, tokenizer, test_df, checkpoint_path)
# test_df["PredictionString"] = test_df["id"].map(predictions)
# test_df[["id", "PredictionString"]].to_csv("submission.csv", index=False)